In [1]:
# Install Requirments (Updated on 12/17/2024)
# !pip3 install -r requirements.txt

In [2]:
import os
import sys

# Suppress TensorFlow logs
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'  # Use asynchronous CUDA memory allocator
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Restrict TensorFlow to GPU 0
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'  # Prevent CUDA errors

# Redirect STDERR to /dev/null to silence C++ warnings
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import tensorflow as tf
import logging

# Suppress TensorFlow and absl logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)

# Clear previous sessions
tf.keras.backend.clear_session()

# GPU configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is configured properly.")
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

sys.stderr = stderr  # Restore STDERR

# Check available GPUs
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


E0000 00:00:1735262566.218507  173723 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735262566.221904  173723 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


No GPU detected. TensorFlow will use the CPU.
Num GPUs Available: 0


In [3]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [13]:
f.predictions('mstr')

MSTR 5m Interval Timestamp: 2024-12-27 03:07:22


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,static
kelly_1:2.5,0.3,0.319403,0.32249,0.191166,0.382069
prob_up,0.104244,0.071749,0.093193,0.0,0.0
prob_static,0.788849,0.828041,0.786716,1.0,1.0
prob_down,0.106907,0.10021,0.120091,0.0,0.0
precision,0.5,0.513859,0.516064,0.422261,0.558621
recall,0.525532,0.512766,0.546809,0.508511,0.344681
f1,0.512448,0.513312,0.530992,0.46139,0.426316
support,"[474.0, 470.0, 470.0]","[474.0, 470.0, 470.0]","[474.0, 470.0, 470.0]","[474.0, 470.0, 470.0]","[474.0, 470.0, 470.0]"


MSTR 15m Interval Timestamp: 2024-12-27 03:07:50


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,static,static,static,static,static
kelly_1:2.5,0.254971,0.274699,0.273224,0.217436,0.27651
prob_up,0.129761,0.080758,0.213987,0.375,0.0
prob_static,0.718577,0.829074,0.596275,0.625,1.0
prob_down,0.151662,0.090168,0.189737,0.0,0.0
precision,0.467836,0.481928,0.480874,0.441026,0.483221
recall,0.509554,0.509554,0.56051,0.547771,0.458599
f1,0.487805,0.495356,0.517647,0.488636,0.470588
support,"[158.0, 157.0, 157.0]","[158.0, 157.0, 157.0]","[158.0, 157.0, 157.0]","[158.0, 157.0, 157.0]","[158.0, 157.0, 157.0]"


MSTR 1h Interval Timestamp: 2024-12-27 03:08:03


...

,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,static,down
kelly_1:2.5,0.118933,0.1376,0.159195,0.114488,0.067017
prob_up,0.343933,0.137167,0.319681,0.125,0.0
prob_static,0.161942,0.106396,0.123965,0.5,0.0
prob_down,0.494125,0.756437,0.556354,0.375,1.0
precision,0.370667,0.384,0.399425,0.367491,0.333584
recall,0.313063,0.324324,0.313063,0.468468,1.0
f1,0.339438,0.351648,0.35101,0.411881,0.500282
support,"[443.0, 444.0, 444.0]","[443.0, 444.0, 444.0]","[443.0, 444.0, 444.0]","[443.0, 444.0, 444.0]","[443.0, 444.0, 444.0]"


MSTR 1wk Interval Timestamp: 2024-12-27 03:09:21


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,down,down,down,down,down
kelly_1:2.5,0.105155,0.057692,0.122388,0.072727,0.061538
prob_up,0.275015,0.279191,0.325515,0.375,0.0
prob_static,0.091462,0.052086,0.139412,0.125,0.0
prob_down,0.633522,0.668723,0.535073,0.5,1.0
precision,0.360825,0.326923,0.373134,0.337662,0.32967
recall,0.388889,0.377778,0.277778,0.288889,1.0
f1,0.374332,0.350515,0.318471,0.311377,0.495868
support,"[91.0, 92.0, 90.0]","[91.0, 92.0, 90.0]","[91.0, 92.0, 90.0]","[91.0, 92.0, 90.0]","[91.0, 92.0, 90.0]"


MSTR 1mo Interval Timestamp: 2024-12-27 03:09:37


,XGBoost,GradientBoosting,RandomForest,KNN,LSTM
prediction,up,up,up,down,static
kelly_1:2.5,0.115789,0.16,0.253333,0.3,0.098305
prob_up,0.811615,0.954768,0.522358,0.375,0.0
prob_static,0.124556,0.010552,0.109187,0.0,1.0
prob_down,0.063829,0.034681,0.368455,0.625,0.0
precision,0.368421,0.4,0.466667,0.5,0.355932
recall,0.368421,0.315789,0.368421,0.157895,1.0
f1,0.368421,0.352941,0.411765,0.24,0.525
support,"[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]","[19.0, 21.0, 19.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')